Importando as Bibliotecas

In [1]:
import MySQLdb
import sqlalchemy
import pandas as pd 
import os

Configuração e conexão com o banco de dados

In [90]:
#SQL Connection 
user = 'jonathas' #usuário
password = '123456' #senha
host = 'localhost' #manter como está
database = 'desafio2IGTI' #schema
string_conexao = f'mysql://{user}:{password}@{host}/{database}' #url de conexão com o bacanco de dados, não mudar.
engine = sqlalchemy.create_engine(string_conexao)
conn = engine.connect()

Criação do df lendo o arquivo .csv

In [59]:
df = []
df = pd.read_csv('base/dados_jogadores.csv', sep = ';', encoding='latin-1') 
df = df.drop_duplicates()

Verifica quais colunas possuem o valor faltando/null

In [74]:
null_columns=df.columns[df.isnull().any()]
null_columns

Index(['num_vitorias', 'num_derrotas', 'total_partidas'], dtype='object')

Retorna quais as linhas possuem o valor null

In [75]:
df[df.isnull().any(axis=1)][null_columns]

,num_vitorias,num_derrotas,total_partidas
46,NaN,68.0,85.0
55,NaN,29.0,80.0
71,29.0,7.0,NaN
86,80.0,6.0,NaN
97,NaN,1.0,10.0
137,NaN,28.0,44.0
246,33.0,NaN,89.0
251,NaN,27.0,28.0
255,1.0,NaN,73.0
287,3.0,NaN,17.0


Verifica se existe alguma linha com mais de um valor null/nan/faltando

In [80]:
#Aqui estamos analisando se uma linha possui um ou mais valor null, no caso, estamos somando as colunas dessa linha que possuem esse esse valor null
#observamos que nenhuma linha possui mais de um valor null, caso existisse algum valor, como nós analisamos apenas as 3 colunas, o valor máximo de cada linha seria 3.
df[df.isnull().any(axis=1)][null_columns].isnull().sum(axis=1)

46     1
55     1
71     1
86     1
97     1
137    1
246    1
251    1
255    1
287    1
323    1
336    1
372    1
407    1
423    1
453    1
458    1
473    1
490    1
492    1
496    1
dtype: int64

E se fossem 10.000 linhas? Como iríamos analisar se existe uma linha com mais de um valor null

In [94]:
tamanho = len(df[df.isnull().any(axis=1)][null_columns].isnull().sum(axis=1)) #tamanho do vetor contendo as linhas com valores null
soma = sum(df[df.isnull().any(axis=1)][null_columns].isnull().sum(axis=1)) #soma dos valores do vetor contendo as linhas com valores null
if(soma == tamanho):
    print('Não existem linhas com mais de um null/nan')
else: 
    print('Existem linhas com mais de um valor null/nan')


Não existem linhas com mais de um null/nan


Teoricamente deveríamos percorrer todas as linhas de cada coluna através de um for para resolver o problema dos valores faltando.
Se observarmos a tabela, veremos que:

1 - Na linha que falta a o num_vitorias , temos num_derrotas e total_partidas. A lógia é se fazer vitorias = total partidas - derrotas.

2 - Na linha que falta a o num_derrotas , temos num_vitorias e total_partidas. A lógia é se fazer derrotas = total partidas - vitorias.

3 - Na linha que falta a o total_partidas , temos num_vitorias e num_derrotas. A lógia é se fazer total_partidas = derrotas + vitorias.

E é justamente o que fazemos aqui no código abaixo.

A função .fillna() vai preencher o local que possui um NaN com o que estiver dentro dos parênteses. Nós nem precisamos utilizar o for, pois o python entende que a operação que estamos fazendo é para todo o dataframe selecionado.

In [1]:
#Aqui fazemos com que a coluna de num_vitorias receba ela mesma preenchida onde tem o valor "NaN" com total_partidas - total_derrotas.
df['num_vitorias'] = df['num_vitorias'].fillna(df['total_partidas']-df['num_derrotas']);

#As duas outras colunas passam pela mesma transformação
df['num_derrotas'] = df['num_derrotas'].fillna(df['total_partidas']-df['num_vitorias']);
df['total_partidas'] = df['total_partidas'].fillna(df['num_vitorias']+df['num_vitorias']);



NameError: name 'df' is not defined

Como no Trabalho 2, criamos uma lista para essas duas colunas únicas afim de inseri-las no banco de dados

In [94]:
paises = list(df.pais.unique());
jogos = list(df.jogo.unique());

Inserindo Países no banco de dados

In [95]:
#Inserir Países
for p in paises:
    try:
        #Nessa primeira etapa, vamos inserir na tabela "bebida" na coluna bebida, o valor de b.
        querey = f"""insert into pais (nome_pais) 
                     values ('{p}')"""
        conn.execute(querey)
    except Exception as e:
        print(f"Não foi possível inserir o regristro {p}. O erro foi encontrado foi: {e}")  
print("Países inseridos")

Países inseridos


Inserindo os jogos no banco de dados

In [96]:
#Inserir Jogos
for j in jogos:
    try:
        #Nessa primeira etapa, vamos inserir na tabela "bebida" na coluna bebida, o valor de b.
        querey = f"""insert into jogo (nome_jogo) 
                     values ('{j}')"""
        conn.execute(querey)
    except Exception as e:
        print(f"Não foi possível inserir o regristro {j}. O erro foi encontrado foi: {e}")  
print("Jogos inseridos")

Jogos inseridos


Aqui lemos as tabelas 'jogo' e 'pais' do banco de dados afim de identificar os valores dos cod_jogo e cod_pais para posterior uso no dicionário.

In [97]:
#Recebenbo dados do banco
df_jogos = pd.read_sql('jogo', con=conn);
df_paises = pd.read_sql('pais', con=conn);

Criando Dicionário dos jogos e países e os substituindo no nosso dataframe.

In [98]:
#Criando dicionario dos jogos
dict_jogos = df_jogos.set_index(df_jogos.nome_jogo)['cod_jogo'].to_dict();
#Substituindo no nosso dataframe
df.jogo = df.jogo.map(dict_jogos);
#Criando dicionário dos países
dict_paises = df_paises.set_index(df_paises.nome_pais)['cod_pais'].to_dict();
#Substituindo no nosso dataframe
df.pais = df.pais.map(dict_paises);

Visualizando nosso dataframe apenas com os códigos dos países e jogos.

In [2]:
df[:5]#primeiras 5 linhas

NameError: name 'df' is not defined

Podemos observar que o formato da data_nascimento não é o formato padrão do python e nem é aceito pelo banco. Portanto precisamos dizer para o python qual o formato que ele se encontra para que a função to_datetime o transforme para o padrão.

Observando a tabela, é possível ver que o formato utilizado é o '%d/%m/%Y'. Utilizamos esse formato dentro da função abaixo para que o python o transforme no padrão Y-m-d.

In [113]:

df.data_nascimento = pd.to_datetime(df['data_nascimento'], format='%d/%m/%Y')
df

,jogador,genero,data_nascimento,jogo,pais,num_vitorias,num_derrotas,total_partidas
0,jogador 1,Masculino,1990-09-03,1,1,14.0,0.0,14.0
1,Jogador 2,Feminino,1986-05-01,2,1,39.0,20.0,59.0
2,Jogador 3,Masculino,1964-10-04,2,2,26.0,3.0,29.0
3,Jogador 4,Masculino,1979-09-28,2,3,9.0,72.0,81.0
4,Jogador 5,Masculino,1978-09-03,1,4,1.0,54.0,55.0
...,...,...,...,...,...,...,...,...
494,Jogador 495,Feminino,1974-06-03,1,1,52.0,28.0,80.0
495,Jogador 496,Masculino,1959-02-09,1,1,13.0,51.0,64.0
496,Jogador 497,Masculino,1962-01-06,2,3,57.0,23.0,80.0
497,Jogador 498,Masculino,1958-09-28,2,1,5.0,3.0,8.0


Armazenamento padrão que já aprendemos nas outras aulas

In [114]:
for j in df.itertuples():
    nome_jogador = j.jogador
    genero = j.genero
    data_nascimento = j.data_nascimento
    cod_jogo = j.jogo
    cod_pais = j.pais
    num_vitorias = j.num_vitorias
    num_derrotas = j.num_derrotas
    total_partidas = j.total_partidas
    try:
        querey = f"""insert into jogador (nome_jogador, genero, data_nascimento, cod_pais, cod_jogo, num_vitorias, num_derrotas, total_partidas)
                     values ('{nome_jogador}',  '{genero}', '{data_nascimento}', '{cod_pais}','{cod_jogo}', '{num_vitorias}', '{num_derrotas}', '{total_partidas}')"""  
        conn.execute(querey) #enviar soliciatação par inserir      
        print(f"Registro inserido com sucesso: {nome_jogador}") #Se ocorrer tudo bem
    except Exception as e:        
        print(f"Não foi possível inserir o regristro {nome_jogador}. O erro foi encontrado foi: {e}") #Se houver algum erro
print("Jogadores Inseridos")

ador 33
Registro inserido com sucesso: Jogador 34
Registro inserido com sucesso: Jogador 35
Registro inserido com sucesso: Jogador 36
Registro inserido com sucesso: Jogador 37
Registro inserido com sucesso: Jogador 38
Registro inserido com sucesso: Jogador 39
Registro inserido com sucesso: Jogador 40
Registro inserido com sucesso: Jogador 41
Registro inserido com sucesso: Jogador 42
Registro inserido com sucesso: Jogador 43
Registro inserido com sucesso: Jogador 44
Registro inserido com sucesso: Jogador 45
Registro inserido com sucesso: Jogador 46
Registro inserido com sucesso: Jogador 47
Registro inserido com sucesso: Jogador 48
Registro inserido com sucesso: Jogador 49
Registro inserido com sucesso: Jogador 50
Registro inserido com sucesso: Jogador 51
Registro inserido com sucesso: Jogador 52
Registro inserido com sucesso: Jogador 53
Registro inserido com sucesso: Jogador 54
Registro inserido com sucesso: Jogador 55
Registro inserido com sucesso: Jogador 56
Registro inserido com suce